# Final Project

Anna Pauxberger

CS110 Final Project

20 April 2018 


## 1. Application: portfolio optimization

**Finding stock combinations with budget constraints:** 
- First, I **scraped data** of the stocks' prices and earnings per share from yahoo finance. No easily available option offers to give data on stocks other than price, but for solving the optimization with the knapsack solution, I had to find a parameter that measures the value of the stock - earnings per share (eps). 
- The data is **stored in heaps**, which allows to efficiently get the minimum/ maximum value. By storing the stock data as a dictionary in the heap, I create a list of a certain number of best performing stocks according to eps, and easily have their stock price and name available. 
- I then run my set of best performing stocks through the **knapsack problem** with prices as weights and eps as values. By backtracking the knapsack solution, I am able to find which stocks I should fill my portfolio with, to maximize value given my budget constraints. 

**Why break it down into heap and knapsack?** Even though my budget is limited, I don't want the cheapest stocks that have low or zero eps. The heap ensures that only the best stocks are taken into account, and the knapsack solution takes care of fitting into my budget.

**Why a heap data structure?** Heap data structures are efficient ways of storing data, keeping track of minimum (maximum) variable and updating the tree. Red-Black-Trees would be optimal if we wanted to keep the stocks in order. For example, if we wanted to know the stocks with smallest and largest value, or searching for the stocks by a certain value could be done in O(log n) time. However, we are only interested in the smallest value. The heap property that holds is such that for all nodes i (excluding the rood), the parent has to be smaller than its children. 
While hash tables would provide retrieval in constant time, they do not provide a sorting option and can thus not be considered.

**Why a 0-1-knapsack problem?** The dynamic programming 0-1 knapsack solution is efficient and optimal. There is a memory trade-off to the recursive solution, but outperforms the recursive solution with regards to time complexity asymptotically. The greedy knapsack solution is not applicable here, as the items are integers and cannot be broken down infinitely.

**What about complexity?**
- A heap takes O(n) to build, and O(lg n) to heapify. Therefore the initial creation of the heap takes O(n lg n).  
- Following heapify() procedures after heappop()ing the root element take only O(lg n).
- The knapsack problem performs at O(nw), with n number of items and w distinct weights

With n=inquired_stocks, m=best_stocks, w=distinct_weights:
- inquiring stocks: O(n)
- building a heap: O(n lg n)
- extracting m minimums: O(m)
- creating knapsack table: O(mw)
- finding knapsack combinations: O(m+w)

O(n lg n) and O(mw) dominate the other complexities. Which of them determines the overall complexity depends on values for n and m, though I would argue that O(mw) will dominate, as long as n is not substantially larger than m. (Looking at the empirical performance of building heaps and finding the knapsack supports this intuition, as the former takes much shorter than the latter.)

**What are limitations?** Stocks have to be indicated and downloaded externally to the application (unfortunately, the yahoo python package is down). Only one share per stock can be added to the portfolio. Share prices are taken as integers instead of floats in order to work with the knapsack problem. Once yahoo finance is online again, heappush and heapdelete could be used to continuously update the heap based on a choice of stocks.


HCs and LOs:
- \#optimization: I apply the 0-1 knapsack problem to optimize a portfolio based on its eps value. The solution itself is optimized by applying dynamic programming to lower complexity.
- \#searchtrees: I apply a heap structure to store and keep in order stock prices (or eps). 

In [1]:
import json
import math
import time as time

# Data pre-processing

In [2]:
# https://www.scrapehero.com/scrape-yahoo-finance-stock-market-data/

# yahoo_finance.py  -> scrape data from yahoo finance
# tickers.json      -> scraped stocks (aapl,goog,nflx,ibm,fb,twtr,tsla,msft,baba,ge)

with open('tickers.json', 'r') as f:
    tickers_dict = json.load(f)

stock_info = []
for ticker in tickers_dict:
    stock_info.append([ticker['ticker'], ticker['Open'], ticker['EPS (TTM)']])

for stock in stock_info:
    oldprice = stock[1]
    newprice = oldprice.replace(",", "")
    price = float(newprice)
    stock[1] = price
    
# Create dictionaries for stocks
names, price, eps = [], [], []
for i in range(len(stock_info)):
    names.append(stock_info[i][0])
    price.append(stock_info[i][1])
    eps.append(stock_info[i][2])
print(f'stocks:{names}')
print(f'prices:{price}')
print(f'eps:{eps}')

def create_dictionary(keys, values1, values2):
    dictionary = {}
    for i in range(len(keys)):
        dictionary[keys[i]] = (values1[i], values2[i])
    return dictionary
        
name_dict = create_dictionary(names, price, eps)
price_dict = create_dictionary(price, names, eps)
eps_dict = create_dictionary(eps, price, names)

stocks:['aapl', 'goog', 'nflx', 'ibm', 'fb', 'twtr', 'tsla', 'msft', 'baba', 'ge']
prices:[174.95, 1069.4, 332.88, 149.19, 166.2, 31.37, 291.08, 96.44, 183.26, 13.67]
eps:[9.7, 17.996, 1.25, 6.135, 5.39, -0.15, -11.833, 1.23, 4.09, -0.719]


# Storing in a heap

In [3]:
# https://www.geeksforgeeks.org/heap-sort/ 
def heapify_1(arr, n, i):
    # n = size of heap, rooted at index i
    largest = i  # Initialize largest as root
    l = 2 * i + 1     # left = 2*i + 1
    r = 2 * i + 2     # right = 2*i + 2
 
    # See if left child of root exists and is
    # greater than root
    if l < n and arr[i] < arr[l]:
        largest = l
 
    # See if right child of root exists and is
    # greater than root
    if r < n and arr[largest] < arr[r]:
        largest = r
 
    # Change root, if needed
    if largest != i:
        arr[i],arr[largest] = arr[largest],arr[i]  # swap
 
        # Heapify the root.
        heapify_1(arr, n, largest)

def heapSort(arr):
    n = len(arr)
 
    # Build a maxheap.
    for i in range(n, -1, -1):
        heapify_1(arr, n, i)
 
    # One by one extract elements
    for i in range(n-1, 0, -1):
        arr[i], arr[0] = arr[0], arr[i]   # swap
        heapify_1(arr, i, 0)
    return arr


def build_max_heap(a):
    heapsize = len(a)
    for i in range(heapsize/2, 1):
        heapify(a)

def heappush(hp, a):
    key = len(hp)
    hp.append(a)
    while key > 0:
        parent = (key - 1) // 2
        if hp[parent] <= hp[key]: 
            break
        hp[key], hp[parent] = hp[parent], hp[key]
        key = parent  
    
def heappop(hp):
    first = hp[0]
    last = hp.pop()
    s = len(hp)
    if s == 0: 
        return first
    hp[0] = last
    key = 0
    while True:
        kid1 = 2 * key + 1
        if kid1 >= s: 
            return first
        kid2 = kid1 + 1
        if kid2 < s and hp[kid2] < hp[kid1]:
            kid = kid2
        else:
            kid = kid1
        if hp[key] <= hp[kid]: 
            return first
        hp[kid], hp[key] = hp[key], hp[kid]
        key = kid

def heapify(hp):
    s=len(hp)
    for i in range((s//2)-1,-1,-1):
        root = hp[i]             
        kid = 2*i+1
        while kid < s:
            if kid < s-1 and hp[kid] > hp[kid+1]:
                kid = kid + 1
            if root <= hp[kid]:     
                break
            hp[(kid-1)//2] = hp[kid]   
            kid = 2*kid+1
        hp[(kid-1)//2] = root      
    return hp

def heapdelete(hp, a):
    a[0] = -math.inf
    heapify(hp)
    heappop(hp)

# Optimizing Knapsack

In [4]:
def knapsack(W, wt, val, n):
    K = [[0 for x in range(W+1)] for x in range(n+1)]
 
    # Build table K[][] in bottom up manner
    for i in range(n+1):
        for w in range(W+1):
            if i==0 or w==0:
                K[i][w] = 0
            elif wt[i-1] <= w:
                K[i][w] = max(val[i-1] + K[i-1][w-wt[i-1]],  K[i-1][w])
            else:
                K[i][w] = K[i-1][w]
 
    return K
 
# Driver program to test above function
val = [60, 100, 120]  #values
wt = [10, 20, 30]     #weights
W = 50                #capacity
n = len(val)          #number of items
testsack = knapsack(W, wt, val, n)

# http://cse.unl.edu/~goddard/Courses/CSCE310J/Lectures/Lecture8-DynamicProgramming.pdf

def knapsack_solution(K,W,val,wt,n):
    knapsack = []
    i=n
    k=W
    
    while i > 0 and k > 0:
        if K[i][k] != K[i-1][k]:
            knapsack.append(i) # mark ith item as in the knapsack
            i = i-1         
            k = k-wt[i]
        else:
            i = i-1         # assuming ith item is not in the knapsack
#     return knapsack
    print(f'max value: {K[n][W]}')
    print(f'optimal combination: {knapsack}')
    return (K[n][W], knapsack)

knapsack_solution(testsack,50,val,wt,len(val))

max value: 220
optimal combination: [3, 2]


(220, [3, 2])

In [5]:
#Example 

val = [9.7, 17.996, 1.25, 6.135]  #values
wt = [174.95, 1069.4, 332.88, 149.19]     #weights

#in order to work with knapsack 01, they have to be integers
val=[int(i) for i in val] 
wt=[int(i) for i in wt]

W = 1500                #capacity
n = len(val)          #number of items

testsack = knapsack(W, wt, val, n)
print(testsack[n][W])

knapsack_solution(testsack,W,val,wt,len(val))

32
max value: 32
optimal combination: [4, 2, 1]


(32, [4, 2, 1])

# Combining heap and knapsack

In [6]:
start_time = time.time()
# take the 5 items from min heap that show highest eps
eps_dict = create_dictionary(eps, price, names)
eps_heap_data = [[-key, value] for key, value in eps_dict.items()]
eps_heap = heapSort(eps_heap_data)

best_stock_eps = []
for i in range(5):
    best_stock_eps.append(heappop(eps_heap))
    heapify(eps_heap)

# and run it through the knapsack based on constraints
middle_time = time.time()
weights = [stock[1][0] for stock in best_stock_eps] # already ordered by smallest value
values = [stock[0] for stock in best_stock_eps]
names = [stock[1][1] for stock in best_stock_eps]

portfolio_size = 1500
weights = [int(i) for i in weights]
values = [abs(int(i)) for i in values] #return to be positive
n = len(values)
print(names)
print(values)

portfolio_table = knapsack(portfolio_size, weights, values, n)
portfolio_optimal = knapsack_solution(portfolio_table, portfolio_size, weights, values, n)
end_time = time.time()

print('')
print('Optimal portfolio:')
for name in portfolio_optimal[1]:
    print(f'1 share of {names[name-1]} at eps {values[name-1]}')

['goog', 'aapl', 'ibm', 'fb', 'baba']
[17, 9, 6, 5, 4]
max value: 32
optimal combination: [3, 2, 1]

Optimal portfolio:
1 share of ibm at eps 6
1 share of aapl at eps 9
1 share of goog at eps 17


In [7]:
print(f'Building the heap: {middle_time - start_time}')
print(f'Optimizing knapsack: {end_time - middle_time}')

Building the heap: 0.0002598762512207031
Optimizing knapsack: 0.007140159606933594
